In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.Dense(64, activation='linear', input_shape=(dset_features.shape[1],)),  # Hidden layer 1
            layers.Dense(32, activation='selu'),  # Hidden layer 2
            layers.Dense(1, activation='linear')  # Output layer
            ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))
        elif layer_type == 'gru':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.GRU(units, return_sequences=return_sequences))
        elif layer_type == 'attention':
            model.add(layers.MultiHeadAttention(num_heads=layer_config[1], key_dim=layer_config[2]))
        elif layer_type == 'cbam':
            model.add(CBAM())
        elif layer_type == 'channel_attention':
            model.add(ChannelAttention())
        elif layer_type == 'linear_attention':
            model.add(LinearAttention())
        elif layer_type == 'nonlinear_attention':
            model.add(NonLinearAttention())

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='multi_layer_perceptron', dataset_name="normal_distribution_dataset", total_epoch=50):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')

    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='multi_layer_perceptron', dataset_name="normal_distribution_dataset",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    correct_predictions = np.sum(np.abs(true_labels - predictions.flatten()) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, predictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, predictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), predictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }

In [2]:
# Load dataset
dataset_name = "normal_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4929 - loss: 0.2901 - val_accuracy: 0.4972 - val_loss: 0.2512
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 994us/step - accuracy: 0.4983 - loss: 0.2515 - val_accuracy: 0.5017 - val_loss: 0.2507
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.5100 - loss: 0.2507 - val_accuracy: 0.5082 - val_loss: 0.2502
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.4922 - loss: 0.2506 - val_accuracy: 0.5038 - val_loss: 0.2503
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - accuracy: 0.5046 - loss: 0.2504 - val_accuracy: 0.5045 - val_loss: 0.2502
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.5093 - loss: 0.2504 - val_accuracy: 0.4997 - val_loss: 0.2508
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - accuracy: 0.5053 - loss: 0.2503 - val_accuracy: 0.4935 - val_loss: 0.2502
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5086 - loss: 0.2503 - val_

In [3]:
# Load dataset
dataset_name = "binomial_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5072 - loss: 2.9083 - val_accuracy: 0.4897 - val_loss: 0.3387
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5014 - loss: 0.3115 - val_accuracy: 0.4897 - val_loss: 0.2720
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5041 - loss: 0.2620 - val_accuracy: 0.4897 - val_loss: 0.2535
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5022 - loss: 0.2514 - val_accuracy: 0.4897 - val_loss: 0.2507
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - accuracy: 0.5037 - loss: 0.2501 - val_accuracy: 0.4897 - val_loss: 0.2503
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.5056 - loss: 0.2500 - val_accuracy: 0.4897 - val_loss: 0.2503
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step - accuracy: 0.5077 - loss: 0.2499 - val_accuracy: 0.4897 - val_loss: 0.2502
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.5061 - loss: 0.2500 - val_accu

In [4]:
# Load dataset
dataset_name = "gamma_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4977 - loss: 0.2916 - val_accuracy: 0.4995 - val_loss: 0.2517
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.4991 - loss: 0.2527 - val_accuracy: 0.5002 - val_loss: 0.2539
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step - accuracy: 0.5017 - loss: 0.2531 - val_accuracy: 0.4945 - val_loss: 0.2505
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.4910 - loss: 0.2529 - val_accuracy: 0.5002 - val_loss: 0.2513
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - accuracy: 0.4996 - loss: 0.2527 - val_accuracy: 0.5067 - val_loss: 0.2503
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 953us/step - accuracy: 0.4983 - loss: 0.2527 - val_accuracy: 0.5005 - val_loss: 0.2526
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.5071 - loss: 0.2516 - val_accuracy: 0.5010 - val_loss: 0.2520
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - accuracy: 0.5115 - loss: 0.2518 - val_accuracy:

In [5]:
# Load dataset
dataset_name = "poisson_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4924 - loss: 0.3048 - val_accuracy: 0.5038 - val_loss: 0.2528
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.4976 - loss: 0.2513 - val_accuracy: 0.4918 - val_loss: 0.2508
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - accuracy: 0.4930 - loss: 0.2504 - val_accuracy: 0.4918 - val_loss: 0.2502
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.4984 - loss: 0.2505 - val_accuracy: 0.4940 - val_loss: 0.2507
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.4983 - loss: 0.2506 - val_accuracy: 0.4940 - val_loss: 0.2512
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5005 - loss: 0.2504 - val_accuracy: 0.5060 - val_loss: 0.2522
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 0.4951 - loss: 0.2505 - val_accuracy: 0.4940 - val_loss: 0.2510
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.5020 - loss: 0.2503 - val_accuracy: 0

In [6]:
# Load dataset
dataset_name = "exponential_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5040 - loss: 0.6723 - val_accuracy: 0.4972 - val_loss: 0.2841
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.5022 - loss: 0.2686 - val_accuracy: 0.4972 - val_loss: 0.2520
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.5082 - loss: 0.2504 - val_accuracy: 0.4972 - val_loss: 0.2501
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.5097 - loss: 0.2499 - val_accuracy: 0.4972 - val_loss: 0.2500
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - accuracy: 0.5006 - loss: 0.2500 - val_accuracy: 0.4972 - val_loss: 0.2501
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - accuracy: 0.5071 - loss: 0.2500 - val_accuracy: 0.4972 - val_loss: 0.2500
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - accuracy: 0.4981 - loss: 0.2500 - val_accuracy: 0.4972 - val_loss: 0.2501
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.5061 - loss: 0.2500 - val_accuracy: